recreating abm

In [1]:
# imports
import numpy as np
import pandas as pd

In [2]:
# colors for each bacteria in the graphics
colors = {"coop": "blue", "cheat": "red"}

In [3]:
# bateria class
class Bacteria:
    def __init__(self, energy, recombination, breed, loc):
        # int for bacteria energy
        self.energy = energy
        # boolean for if it take part in recombination
        self.recombination = recombination
        # int for breed
        self.breed = breed
        # list of ints for location [x, y]
        self.loc = loc
        # string for the color according to the breed
        self.color = colors[breed]
    
    def set_breed(self, breed):
        self.breed = breed
        self.color = colors[breed]

    def get_move_options(self, size):
        options = []
        if self.loc[0] != 0:
            options.append([self.loc[0] - 1, self.loc[1]])
        if self.loc[0] != size[0] - 1:
            options.append([self.loc[0] + 1, self.loc[1]])
        if self.loc[1] != 0:
            options.append([self.loc[0], self.loc[1] - 1])
        if self.loc[1] != size[1] - 1:
            options.append([self.loc[0], self.loc[1] + 1])
        return options

In [39]:
# simulation class - we can remove print statements, only for testing
# TODO: add tracking and return of simulation data over generations
# TODO: add animation of simulation based on tracking of data over generations
class Sim:
    def __init__(self, start_energy, population_size, population_viscosity, recombination_cost,
                 mutation_rate, recombination_rate, percent_recombination, percent_cooperation,
                 contribution, multiplier, max_x, max_y):
        self.start_energy = start_energy
        self.population_size = population_size
        self.population_viscosity = population_viscosity
        self.recombination_cost = recombination_cost
        self.mutation_rate = mutation_rate
        self.recombination_rate = recombination_rate
        self.percent_recombination = percent_recombination
        self.percent_cooperation = percent_cooperation
        self.contribution = contribution
        self.multiplier = multiplier
        self.max_x = max_x
        self.max_y = max_y

        self.bacteria = []

        self.num_rec = int(np.round(population_size * percent_recombination))
        self.num_coop = int(np.round(population_size * percent_cooperation))

        np.random.seed(0)
        index_rec = np.random.choice(population_size, self.num_rec, replace=False)
        index_coop = np.random.choice(population_size, self.num_coop, replace=False)

        for i in np.arange(population_size):
            bac = Bacteria(start_energy, True if i in index_rec else False, "coop" if i in index_coop else "cheat",
                           [np.random.choice(max_x, 1)[0], np.random.choice(max_y,1)[0]])
            self.bacteria.append(bac)
    
    def simulate_gen(self, gen):
        print("starting gen", gen)
        # coops contribute
        def contribute(bac):
            if bac.breed == "coop":
                bac.energy -= self.contribution
            return bac
        self.bacteria = list(map(contribute, self.bacteria))

        # all benifit
        def benifit(bac, amt):
            bac.energy += amt
            return bac
        self.bacteria = list(map(benifit, self.bacteria, [(self.num_coop * self.multiplier) / len(self.bacteria)]
                                 * len(self.bacteria)))
        
        # lose fitness
        def update_fitness(bac):
            if np.random.choice(2, 1) == 1:
                bac.energy -= 1
            if bac.recombination:
                bac.energy -= self.recombination_cost
            return bac
        self.bacteria = list(map(update_fitness, self.bacteria))

        # bacteria die if fitness is < 0
        alive_bac = []
        for b in self.bacteria:
            if b.energy < 0:
                if b.breed == "coop":
                    self.num_coop -= 1
                if b.recombination:
                    self.num_rec -= 1
            else:
                alive_bac.append(b)
        self.bacteria = alive_bac
        if len(self.bacteria) == 0:
            return

        # repopulation
        while len(self.bacteria) < self.population_size:
            parent = self.bacteria[np.random.choice(len(self.bacteria), 1)[0]]
            mutate = np.random.choice(2, 1, p=[1 - self.mutation_rate, self.mutation_rate])[0]
            if mutate == 1:
                breed = parent.breed
            elif parent.breed == "coop":
                breed = "cheat"
            else:
                breed = "coop"
            if breed == "coop":
                self.num_coop += 1
            if parent.recombination:
                self.num_rec += 1
            bac = Bacteria(parent.energy, parent.recombination, breed, parent.loc)
            self.bacteria.append(bac)

        # conversion/recombination
        if self.num_rec != 0:
            # bacterias in each location
            bac_locs = {}
            # number of recombination bacterias in each location
            bac_rec = {}
            for b in self.bacteria:
                if str(b.loc) in bac_locs.keys():
                    bac_locs[str(b.loc)].append(b)
                    if b.recombination:
                        bac_rec[str(b.loc)] += 1
                else:
                    bac_locs[str(b.loc)] = [b]
                    if b.recombination:
                        bac_rec[str(b.loc)] = 1
                    else:
                        bac_rec[str(b.loc)] = 0
            for k in bac_locs.keys():
                if bac_rec[k] == 0 or bac_rec[k] == len(bac_locs[k]):
                    continue
                for b in bac_locs[k]:
                    if not b.recombination:
                        # pick whether to recombine or not for eah recombination bacteria
                        recombine = np.random.choice(2, bac_rec[k], p=[1-self.recombination_rate, self.recombination_rate])
                        if np.sum(recombine) > 0:
                            b.recombination = True
                            self.num_rec += 1

        # movement
        if gen % self.population_viscosity == 0:
            for b in self.bacteria:
                move_ops = b.get_move_options([self.max_x, self.max_y])
                b.loc = move_ops[np.random.choice(len(move_ops), 1)[0]]
        
    def simulate(self, gens):
        print("starting simulation")
        for g in np.arange(gens):
            self.simulate_gen(g)
            if len(self.bacteria) == 0:
                print("all dead")
                break
            print("***results after gen***", g)
            for b in self.bacteria:
                print(b.recombination)
                print(b.breed)
                print(b.energy)
                print(b.loc)

In [40]:
# testing of current simulation code
sim = Sim(10, 3, 2, 1, 0.5, 0.5, 0.5, 0.5, 1, 2, 5, 10)
print("***before***")
for b in sim.bacteria:
    print("*bac*")
    print("rec", b.recombination)
    print("breed", b.breed)
    print("energy", b.energy)
    print("loc", b.loc)
sim.simulate(10)
print("***after***")
for b in sim.bacteria:
    print("*bac*")
    print("rec", b.recombination)
    print("breed", b.breed)
    print("energy", b.energy)
    print("loc", b.loc)

***before***
*bac*
rec False
breed coop
energy 10
loc [np.int64(3), np.int64(3)]
*bac*
rec True
breed cheat
energy 10
loc [np.int64(1), np.int64(3)]
*bac*
rec True
breed coop
energy 10
loc [np.int64(2), np.int64(4)]
starting simulation
starting gen 0
***results after gen*** 0
False
coop
9.333333333333334
[np.int64(2), np.int64(3)]
True
cheat
10.333333333333334
[np.int64(0), np.int64(3)]
True
coop
9.333333333333334
[np.int64(2), np.int64(3)]
starting gen 1
***results after gen*** 1
True
coop
8.666666666666668
[np.int64(2), np.int64(3)]
True
cheat
10.666666666666668
[np.int64(0), np.int64(3)]
True
coop
7.666666666666668
[np.int64(2), np.int64(3)]
starting gen 2
***results after gen*** 2
True
coop
8.000000000000002
[np.int64(3), np.int64(3)]
True
cheat
10.000000000000002
[np.int64(0), np.int64(2)]
True
coop
6.000000000000002
[np.int64(1), np.int64(3)]
starting gen 3
***results after gen*** 3
True
coop
6.333333333333336
[np.int64(3), np.int64(3)]
True
cheat
10.333333333333336
[np.int64(0),

regeneration of results from paper

In [ ]:
# TODO: generate simulations accross a variety of params
# TODO: plot data + show simulation graphics for each independent simulation
# TODO: save end data in dictionary of simulation perfomances for future graphics

In [ ]:
# TODO: create graphics for findings accross all simulations using data from dictionary

simulating in vivo data

In [ ]:
# TODO: generate simulations that match in vivo data
# TODO: plot data + show simulation graphics for each independent simulation

expansion of our simulation model

In [ ]:
# TODO: possibly add another factor to the simulation or another proccess to take place in each gen